## Варіант 5
 ### Завдання  1

 Побудувати кубічний сплайн для таблично заданої функції, з граничними умовами для першої похідної.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import display, Markdown, Math
from matplotlib.pyplot import figure
from numpy.polynomial import polynomial

def init_fig():
    global figure
    figure(num=None, figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;margin:0px auto;}
.tg td{border-style:solid;border-width:0px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;
  padding:10px 5px;word-break:normal;}
.tg th{border-style:solid;border-width:0px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<tbody>
  <tr>
    <td class="tg-c3ow">$i$</td>
    <td class="tg-c3ow">0</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">2</td>
    <td class="tg-c3ow">3</td>
    <td class="tg-0pky">$S'$</td>
  </tr>
  <tr>
    <td class="tg-c3ow">$X$</td>
    <td class="tg-c3ow">-4</td>
    <td class="tg-c3ow">1</td>
    <td class="tg-c3ow">3</td>
    <td class="tg-c3ow">7</td>
    <td class="tg-0pky">$S'(a)=-1$</td>
  </tr>
  <tr>
    <td class="tg-0lax">$Y$</td>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">5</td>
    <td class="tg-0lax">-3</td>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">$S'(b)=2$</td>
  </tr>
</tbody>
</table>

Побудова сплайну буде проводитись за першим способом з файлу, що був надісланий ("КУБІЧНА СПЛАЙН -НТЕРПОЛЯЦІЯ.pdf")

Сплайн матиме вигляд $S(x)=a_{i}+b_{i}(x-x_{i-1})+c_{i}(x-x_{i-1})^{2}+d_{i}(x-x_{i-1})^{3}$

В завданні задані граничні перші похідні, отже будуть використовуватися крайові умови першого типу в точках $x_0$ та $x_N$.

$$S^{\prime}\left(x_{0}\right)=f^{\prime}\left(x_{0}\right) ; \quad S^{\prime}\left(x_{N}\right)=f^{\prime}\left(x_{N}\right)$$

$$\left\{\begin{array}{l}b_{1}=f^{\prime}\left(x_{0}\right) \\ b_{N}+2 c_{N} h_{N}+3 d_{N} h_{n}^{2}=f^{\prime}\left(x_{N}\right)\end{array}\right.$$

Для знаходження коефіцієнтів $c_i$ треба знайти розв'язок наступної СЛАР

$$
\left\{\begin{array}{l}
2 c_{1}+c_{2}=3 \dfrac{y_{1}-y_{0}}{h_{1}^{2}}-\dfrac{3}{h_{1}} f^{\prime} x_{0} \\
h_{i-1} c_{i-1}+2 c_{i}\left(h_{i}+h_{i-1}\right)+h_{i} c_{i+1}=3\left[\dfrac{y_{i}-y_{i-1}}{h_{i}}-\dfrac{y_{i-1}-y_{i-2}}{h_{i-1}}\right], \quad i=\overline{2, N-1} \\
\dfrac{2}{3} h_{N-1} c_{N-1}+\left(h_{N}+\dfrac{4}{3} h_{N-1}\right) c_{N}=-f^{\prime}\left(x_{N}\right)-2 \dfrac{y_{N-1}-y_{N-2}}{h_{N-1}}+3 \dfrac{y_{N}-y_{N-1}}{h_{N}}
\end{array}\right.
$$

Знаходження решти коефіцієнтів буде відбуватись згідно системи нижче
$$
\left\{\begin{array}{l}
a_{i}=y_{i-1}, \quad i=\overline{1, N}\\
b_{i}=\dfrac{y_{i}-y_{i-1}}{h_{i}}-\dfrac{h_{i}}{3}\left(c_{i+1}+2 c_{i}\right), \quad i=\overline{1, N-1} \\
b_{N}=\dfrac{y_{N-1}-y_{N-2}}{h_{N-1}}+\dfrac{h_{N-1}}{3}\left(2 c_{N}+c_{N-1}\right) \\
d_{i}=\dfrac{c_{i+1}-c_{i}}{3 h_{i}}, \quad i=\overline{1, N-1} \\
d_{N}=\dfrac{f^{\prime}\left(x_{n}\right)-b_{N}-2 c_{N} h_{N}}{3 h_{N}^{2}} \\
\end{array}\right.
$$

$h_i$ - різниці $x_i - x_{i-1}, i=\overline{1, N}$